In [9]:
import pandas as pd
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

In [10]:
def extrair_partidas(url, data):
    service = Service()
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    driver = webdriver.Chrome(service=service, options=options)
    time.sleep(5)
    driver.get(url)

    time.sleep(10)
    partidas = driver.find_elements(By.CLASS_NAME, 'tournament-matches-row')
    lista_partidas = []

    for i in range(len(partidas)):
        partida = {}
        linhas = partidas[i].text.split('\n')
        try:
            partida['data'] = data
            partida['time_casa'] = linhas[4]
            partida['player_casa'] = linhas[5]
            partida['gols_casa'] = int(linhas[6])
            partida['gols_fora'] = int(linhas[8])
            partida['player_fora'] = linhas[11]
            partida['time_fora'] = linhas[10]
            partida['total_gols'] = partida['gols_casa'] + partida['gols_fora']

            total = partida['total_gols']
            partida['+ 0.5'] = 'Sim' if total > 0.5 else 'Não'
            partida['+ 1.5'] = 'Sim' if total > 1.5 else 'Não'
            partida['+ 2.5'] = 'Sim' if total > 2.5 else 'Não'
            partida['+ 3.5'] = 'Sim' if total > 3.5 else 'Não'

            if partida['gols_casa'] > partida['gols_fora']:
                partida['vencedor'] = 'Casa'
            elif partida['gols_casa'] < partida['gols_fora']:
                partida['vencedor'] = 'Fora'
            else:
                partida['vencedor'] = 'Empate'

            lista_partidas.append(partida)
        except Exception:
            continue

    driver.close()
    driver.quit()
    return lista_partidas

In [11]:
def extrair_torneios(url):
    service = Service()
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    time.sleep(10)
    links = driver.find_elements(By.CLASS_NAME, 'icon-link')
    lista_torneios = [link.get_attribute('href') for link in links if link.get_attribute('href')]
    driver.close()
    driver.quit()
    return lista_torneios



In [12]:
def salvar(dados):
    for i in range(len(dados)):
        with open('../data/raw/base_jogos.csv', 'a', newline='', encoding='utf-8') as arquivo:
            escrever = csv.writer(arquivo)
            escrever.writerow([
                dados[i]['data'], dados[i]['time_casa'], dados[i]['player_casa'],
                dados[i]['gols_casa'], dados[i]['gols_fora'], dados[i]['player_fora'],
                dados[i]['time_fora'], dados[i]['total_gols'], dados[i]['+ 0.5'],
                dados[i]['+ 1.5'], dados[i]['+ 2.5'], dados[i]['+ 3.5'], dados[i]['vencedor']
            ])

In [13]:
# Exemplo de uso manual (pode ser executado como script ou via %run)
if __name__ == "__main__":
    datas = [(datetime(2025, 8, 1) + timedelta(days=n)).strftime('%Y-%m-%d') 
              for n in range((datetime(2025, 8, 1) - datetime(2025, 8, 1)).days + 1)]
    for data in datas:
        url = f"https://football.esportsbattle.com/en/schedule?page=1&dateFrom={data.replace('-', '/')}+08%3A00&dateTo={data.replace('-', '/')}+23%3A59"
        # url = f"https://football.esportsbattle.com/api/tournaments?page=1&dateFrom={data.replace('-', '/')}+11%3A00&dateTo={data.replace('-', '/')}+02%3A59"
        torneios = extrair_torneios(url)
        for link in torneios:
            partidas = extrair_partidas(link, data)
            salvar(partidas)


FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/base_jogos.csv'

In [ ]:
extrair_torneios(url)


KeyboardInterrupt

